Primero vamos a proceder importando las librerias que vamos a usar en el desarrollo de la práctica

In [65]:
import glob
import cv2
from skimage import io, transform, color
from skimage.exposure import match_histograms
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

Para comenzar con el preprocesamiento de las imagenes primero vamos a clonar el repositorio en el que estan las imagenes para poder trabajar en local.

In [61]:
os.system("git clone https://github.com/Kekon130/ML_Practica3.git") 

32768

Ahora vamos a meter las imagenes en un array para poder acceder de forma fácila a ellas.

In [81]:
ruta_train = "./ML_Practica3/train"
ruta_test = "./ML_Practica3/test"

df_train = []
img_size = 80

for img in os.listdir(ruta_train):
  img_normal = io.imread(os.path.join(ruta_train, img))
  img_resize = transform.resize(img_normal, output_shape=(img_size, img_size))
  
  //img_gray = color.rgb2gray(img_resize)
  
  df_train.append(img_normal)
  

Igualamos los histogramas de todas las imágenes para que sea más homogéneo

He hecho una prueba a ver qué salía con una

In [ ]:
referencia = df_train[5]
imagen_de_prueba = df_train[1]

match = match_histograms (imagen_de_prueba, referencia, multichannel = True)
match_escala_de_grises = color.rgb2gray(match)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(8, 3), sharex=True, sharey=True)
for aa in (ax1, ax2, ax3, ax4):
    aa.set_axis_off()

ax1.imshow(imagen_de_prueba)
ax1.set_title('Original')
ax2.imshow(referencia)
ax2.set_title('Referencia')
ax3.imshow(match)
ax3.set_title('Ajustada')
ax4.imshow(match_escala_de_grises)
ax4.set_title('Ajustada en escala de grises')

plt.tight_layout()
plt.show()

io.imshow(match_escala_de_grises)
